# Miniproject-DQN Notebook
### Paul Boulenger, Paul Devianne

## Part 1: Introduction

## Part 2: Professor Russo's Policy

## Part 3: A Deep Q-learning approach

## Part4: Dealing with a more complex action Space

## Part5: Wrapping Up